In [66]:
import numpy as np

#read txt
amazon = open('C:/Users/james/sentiment labelled sentences/amazon_cells_labelled.txt',"r",encoding="utf-8")
imdb = open('C:/Users/james/sentiment labelled sentences/imdb_labelled.txt',"r",encoding="utf-8")
yelp = open('C:/Users/james/sentiment labelled sentences/yelp_labelled.txt',"r",encoding="utf-8")
data = []
data1 = list(amazon)
data1 += (list(imdb))
data1 += (list(yelp))
#convert to list
for i in data1:
    x = i.split('\t')
    k = x[1].split('\n')    
    data.append([x[0],int(k[0])])

print(len(data))
print(data[:10])


3000
[['So there is no way for me to plug it in here in the US unless I go by a converter.', 0], ['Good case, Excellent value.', 1], ['Great for the jawbone.', 1], ['Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!', 0], ['The mic is great.', 1], ['I have to jiggle the plug to get it to line up right to get decent volume.', 0], ['If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one.', 0], ['If you are Razr owner...you must have this!', 1], ['Needless to say, I wasted my money.', 0], ['What a waste of money and time!.', 0]]


In [112]:
import pandas as pd
import helpers.data_mining_helpers as dmh

data = np.array(data)

df = pd.DataFrame({'sentence': data[:,0],
                   'score': data[:,1],})
print(df)

                                               sentence score
0     So there is no way for me to plug it in here i...     0
1                           Good case, Excellent value.     1
2                                Great for the jawbone.     1
3     Tied to charger for conversations lasting more...     0
4                                     The mic is great.     1
...                                                 ...   ...
2995  I think food should have flavor and texture an...     0
2996                           Appetite instantly gone.     0
2997  Overall I was not impressed and would not go b...     0
2998  The whole experience was underwhelming, and I ...     0
2999  Then, as if I hadn't wasted enough of my life ...     0

[3000 rows x 2 columns]


In [113]:
df.iloc[::100, 0:2][0:10]

,sentence,score
0,So there is no way for me to plug it in here i...,0
100,Integrated seamlessly with the Motorola RAZR p...,1
200,The color is even prettier than I thought it w...,1
300,Sending it back.,0
400,what a disappointment,0
500,"The bose noise cancelling is amazing, which is...",1
600,Their Research and Development division obviou...,1
700,"Also, the phone doesn't seem to accept anythin...",0
800,"Good , works fine.",1
900,"This was utterly confusing at first, which cau...",0


In [114]:
df.loc[::50, 'sentence'][0:10]

0      So there is no way for me to plug it in here i...
50     Not loud enough and doesn't turn on like it sh...
100    Integrated seamlessly with the Motorola RAZR p...
150                               I Was Hoping for More.
200    The color is even prettier than I thought it w...
250    It's fits like a glove and is strong, secure, ...
300                                     Sending it back.
350                             Were JERKS on the phone.
400                                what a disappointment
450    i'm glad i found this product on amazon it is ...
Name: sentence, dtype: object

In [115]:
df.isnull()

,sentence,score
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
2995,False,False
2996,False,False
2997,False,False
2998,False,False


In [116]:
df.isnull().apply(lambda x: dmh.check_missing_values(x))

,sentence,score
0,The amoung of missing records is:,The amoung of missing records is:
1,0,0


In [117]:
dummy_series = pd.Series(["dummy_record"], index=["sentence"])
dummy_series

sentence    dummy_record
dtype: object

In [118]:
result = df.append(dummy_series, ignore_index=True)
result.isnull().apply(lambda x: dmh.check_missing_values(x))

,sentence,score
0,The amoung of missing records is:,The amoung of missing records is:
1,0,1


In [119]:
result.dropna(inplace=True)
result.isnull().apply(lambda x: dmh.check_missing_values(x))

,sentence,score
0,The amoung of missing records is:,The amoung of missing records is:
1,0,0


In [120]:
print(sum(df.duplicated('sentence')))
print(len(df))


17
3000


In [122]:
df.drop_duplicates(keep='first', inplace=True) # keep first
print(len(df))

2983


In [123]:
df_sample = df.sample(n=1000)
print(len(df_sample))

1000
